In [1]:
import itertools
import numpy as np
np.random.seed(42)

In [2]:
class TSPParticleSwarm:
  def __init__(self, n, population_size, alpha, beta):
    self.n = n
    self.size = population_size
    self.alpha = alpha
    self.beta = beta
    self.nodes = np.arange(self.n)
    self.graph = [[0 if i == j else np.random.randint(20)+1 for i in range(self.n)] for j in range(self.n)]
    self.__init_population()

  def __fitness(self, x):
    fitness = 0
    for i in range(self.n):
      j = (i + 1) % self.n
      fitness += self.graph[x[i]][x[j]]
    return fitness

  def __get_gbest(self):
    gbest = self.pbest[0]
    for p in self.pbest:
      if self.__fitness(p) < self.__fitness(gbest):
        gbest = p
    return gbest

  def __init_population(self):
    self.population = [np.random.permutation(self.n) for _ in range(self.size)]
    self.velocity = [[tuple(np.random.choice(self.nodes, 2)) for _ in range(np.random.randint(self.n))] for _ in range(self.size)]
    # self.velocity = [[] for _ in range(self.size)]
    self.pbest = self.population.copy()
    self.gbest = self.__get_gbest()

  def add(self, x, ss):
    x = x.copy()
    for s in ss:
      x[s[0]], x[s[1]] = x[s[1]], x[s[0]]
    return x

  def sub(self, x, y, prob = 1):
    y = y.copy()
    ss = []
    for i in range(len(x)):
      j = np.where(y == x[i])[0][0]
      if np.random.rand() <= prob and i != j:
        ss.append((i, j))
        y[i], y[j] = y[j], y[i]
    return ss

  def __update_pbest(self):
    for i in range(self.size):
      if self.__fitness(self.population[i]) < self.__fitness(self.pbest[i]):
        self.pbest[i] = self.population[i].copy()

  def __reduce(self, ss):
    start = np.arange(self.n)
    end = self.add(start, ss)
    return self.sub(end, start)

  def __update_velocity(self):
    for i in range(self.size):
      self.velocity[i] = self.velocity[i] + self.sub(self.pbest[i], self.population[i], self.alpha) + self.sub(self.gbest, self.population[i], self.beta)
      self.velocity[i] = self.__reduce(self.velocity[i])

  def __update_population(self):
    for i in range(self.size):
      self.population[i] = self.add(self.population[i], self.velocity[i])

  def __perform_iteration(self):
    self.__update_pbest()
    self.gbest = self.__get_gbest()
    self.__update_velocity()
    self.__update_population()

  def display_population(self):
    for p in self.population:
      print(f'SEQUENCE: {p}, FITNESS: {self.__fitness(p)}')
    print()
    print(f'BEST PARTICLE: {self.gbest}, FITNESS: {self.__fitness(self.gbest)}')
    print()

  def get_graph(self):
    return self.graph

  def fit(self, iterations):
    print('INITIAL POPULATION:')
    self.display_population()
    for i in range(iterations):
      self.__perform_iteration()
      print(f'ITERATION {i+1}:')
      self.display_population()

  def best_solution(self):
    all_solutions = list(itertools.permutations(self.nodes))
    best_sol = all_solutions[0]
    for solution in all_solutions:
      if self.__fitness(solution) < self.__fitness(best_sol):
        best_sol = solution
    print(f'BEST SOLUTION: {best_sol}, FITNESS: {self.__fitness(best_sol)}')

In [3]:
population = TSPParticleSwarm(8, 16, 0.7, 0.8)

In [4]:
population.display_population()

SEQUENCE: [0 2 6 3 7 1 4 5], FITNESS: 80
SEQUENCE: [4 2 1 0 6 7 3 5], FITNESS: 65
SEQUENCE: [2 0 5 3 6 1 7 4], FITNESS: 69
SEQUENCE: [1 2 5 0 4 6 7 3], FITNESS: 87
SEQUENCE: [2 1 7 3 5 6 0 4], FITNESS: 73
SEQUENCE: [1 3 4 0 5 2 6 7], FITNESS: 74
SEQUENCE: [3 7 5 6 1 4 2 0], FITNESS: 72
SEQUENCE: [2 4 5 1 7 0 6 3], FITNESS: 92
SEQUENCE: [3 0 1 2 4 5 7 6], FITNESS: 67
SEQUENCE: [3 6 1 5 4 7 0 2], FITNESS: 94
SEQUENCE: [5 3 6 7 2 1 4 0], FITNESS: 53
SEQUENCE: [0 6 5 2 4 3 7 1], FITNESS: 76
SEQUENCE: [2 7 1 5 4 3 0 6], FITNESS: 88
SEQUENCE: [5 1 3 0 2 4 7 6], FITNESS: 95
SEQUENCE: [5 0 4 1 3 7 2 6], FITNESS: 89
SEQUENCE: [6 2 1 7 3 4 0 5], FITNESS: 78

BEST PARTICLE: [5 3 6 7 2 1 4 0], FITNESS: 53



In [5]:
population.fit(20)

INITIAL POPULATION:
SEQUENCE: [0 2 6 3 7 1 4 5], FITNESS: 80
SEQUENCE: [4 2 1 0 6 7 3 5], FITNESS: 65
SEQUENCE: [2 0 5 3 6 1 7 4], FITNESS: 69
SEQUENCE: [1 2 5 0 4 6 7 3], FITNESS: 87
SEQUENCE: [2 1 7 3 5 6 0 4], FITNESS: 73
SEQUENCE: [1 3 4 0 5 2 6 7], FITNESS: 74
SEQUENCE: [3 7 5 6 1 4 2 0], FITNESS: 72
SEQUENCE: [2 4 5 1 7 0 6 3], FITNESS: 92
SEQUENCE: [3 0 1 2 4 5 7 6], FITNESS: 67
SEQUENCE: [3 6 1 5 4 7 0 2], FITNESS: 94
SEQUENCE: [5 3 6 7 2 1 4 0], FITNESS: 53
SEQUENCE: [0 6 5 2 4 3 7 1], FITNESS: 76
SEQUENCE: [2 7 1 5 4 3 0 6], FITNESS: 88
SEQUENCE: [5 1 3 0 2 4 7 6], FITNESS: 95
SEQUENCE: [5 0 4 1 3 7 2 6], FITNESS: 89
SEQUENCE: [6 2 1 7 3 4 0 5], FITNESS: 78

BEST PARTICLE: [5 3 6 7 2 1 4 0], FITNESS: 53

ITERATION 1:
SEQUENCE: [2 4 6 7 5 1 3 0], FITNESS: 102
SEQUENCE: [5 3 6 7 2 1 4 0], FITNESS: 53
SEQUENCE: [7 1 6 0 3 5 4 2], FITNESS: 86
SEQUENCE: [2 0 1 7 3 4 6 5], FITNESS: 90
SEQUENCE: [5 1 3 6 7 4 2 0], FITNESS: 73
SEQUENCE: [1 4 2 7 6 5 3 0], FITNESS: 71
SEQUENCE: [5 3 6

In [6]:
population.best_solution()

BEST SOLUTION: (0, 5, 2, 1, 6, 3, 7, 4), FITNESS: 38
